In [1]:
# importing all the required modules

from importlib import reload
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import os
import errno
import string
from nltk.corpus import reuters
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.text import TextCollection
import collections
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from prettytable import PrettyTable
import time
from datetime import timedelta
from sklearn.metrics import recall_score,precision_score,average_precision_score,f1_score
from sklearn.metrics import accuracy_score,roc_auc_score,mean_absolute_error,cohen_kappa_score
from sklearn.preprocessing import label_binarize,LabelEncoder
from imblearn.metrics import geometric_mean_score
from sklearn import metrics
from sklearn.metrics.cluster import homogeneity_score,completeness_score
import statistics
import math
import sklearn.metrics 
from sklearn.model_selection import KFold,StratifiedKFold
import csv
from sklearn.model_selection import train_test_split
from collections import Counter
import math
from scipy.spatial import distance
import statistics
from scipy.stats import pearsonr,entropy,mode
from scipy.io import arff
from sklearn.metrics import pairwise_distances
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestNeighbors
from sklearn.datasets import load_iris
import warnings
import random
from statistics import mode
random.seed(0)
warnings.filterwarnings('ignore')
import os 
os.environ['R_HOME'] = 'C:\Program Files\R\R-4.3.2'
import rpy2.robjects as robjects
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()


def loadNonIRDatasets(dataset):
    documents,labels=[],[]
    fname=dataset+'.csv'
    with open(fname, 'r') as file:
        reader = csv.reader(file, delimiter = ',')
        for row in reader:
            if len(row)>0:
                if '?' in row:
                    continue
                doc=row[0:len(row)-2]
                try:
                    doc=[abs(float(d)) for d in doc]
                except:
                    continue
                documents.append(doc)
                #print(row)
                labels.append(row[len(row)-1])

    return documents,labels

def Cosine(a, b):#distance
    return distance.cosine(a,b)

def Euclidean(a, b):#distance
    return distance.euclidean(a,b)

def most_common(lst):
    return max(set(lst), key=lst.count)
def PrintDetails(metric="smtp",measure= "Accuracy",Arr=None,kList=[5,15,30,45],train_time=0,test_time=0,std=[]):
    x = PrettyTable()
    #kList=[1,3,5,9,15,30,45]#,70,90,120]
    #if std==[]:
    if len(std)==0:
        x.field_names = ["k",measure,"Train Time","Test time","Total Time"]
    else:
        x.field_names = ["Split",measure,"std","Train Time","Test time","Total Time"]
        
    print("metric",metric)
    print("measure",measure)
    tables.write(("metric\t"+str(metric)+"\n").encode())
    tables.write(("measure\t"+str(measure)+"\n").encode())
    average=0
    total_time=np.add(train_time,test_time)
    for i in range(0,len(Arr)):
        print(train_time[i])
        tr_time=str(timedelta(seconds=(train_time[i])))
        ts_time=str(timedelta(seconds=(test_time[i])))
        
        tot_time=str(timedelta(seconds=(total_time[i])))
        if len(std)==0:
            x.add_row([str(kList[i]),round(Arr[i],4),tr_time,ts_time,tot_time]) 
        else:
            x.add_row([str(kList[i]),round(Arr[i],4),round(std[i],4),tr_time,ts_time,tot_time]) 
            
        average+=Arr[i]
    if len(std)==0:
        x.add_row(["Average",round((average/len(Arr)),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),np.std(train_time),
                   np.std(test_time),np.std(total_time)])   
    else:
        x.add_row(["Average",round((average/len(Arr)),4),round(np.mean(std),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),round(np.std(std),4),
                   round(np.std(train_time),10),
                   round(np.std(test_time),10),
                   round(np.std(total_time),10)])
        
    x.padding_width =1
    tables.write(str(x).encode())
    print(x)

def findDI(train_data,train_labels,metric,k):
    clss=set(train_labels)
    func=globals()[metric]
    radius={}
    th=0.5
    for cl in clss:
        indices=[i for i in range(0,len(train_labels)) if train_labels[i]==cl]
        elements=train_data[indices,:]
        distMat=pairwise_distances(elements,elements,metric=func)
        radius[cl]=np.sum(distMat)/(distMat.shape[0]+distMat.shape[1])
    weights=[]
   
    classifier = NearestNeighbors(n_neighbors=k,metric=func)
    if k>len(train_labels):
         classifier = NearestNeighbors(n_neighbors=len(train_labels),metric=func)
        
    classifier.fit(train_data,train_labels)
    distances,neighVals=classifier.kneighbors(train_data,return_distance=True)
    print(neighVals.shape)
    for i in range(0,train_data.shape[0]):
        r=radius[train_labels[i]]
        ID=0
        neighs=np.array(neighVals[i,:],dtype=int)
        #print(neighs)
        neighLabels = [train_labels[i] for i in neighs]
           
        #neighLabels = [train_labels[i] for i in int(neighVals[i,:])]
        val=0
        c=1
        for nn,neigh in enumerate(neighLabels):
            if distances[i,nn]<r:
                c=c+1
                if neigh==train_labels[i]:
                    val=val+1
        ID=(val/c)
        ##############find weights using validity ###############
        #w=np.sum(val*(1/(distances[i]+alpha)))
        weights.append(ID)
    return weights

def classify(k=3,metric="euclidean",termOccurance=None, docOccurance=None):
    accuracy=[]
    precision=[]
    Recall=[]
    fMeasure=[]
    mprecision=[]
    mRecall=[]
    mfMeasure=[]
    roc=[]
    y_pred=[]
    mae=[]
    kappa=[]
    averageMeanPrecison=[]
    func=globals()[metric]
    kList=[5,15,30,45]
    theta=1
    var=np.zeros(train_data.shape[1])
    train_time=[]
    test_time=[]
    M=5
    alpha=0.5
    for k in kList:
        time1=time.time()
        robjects.r.source('kCNN.R')
        time2=time.time()
        pred,temp = robjects.r['kcnn'](train=train_data, test=test_data,cl=train_labels, k = k)
        pred = [eval(ii) for ii in pred]      
        y_pred.append(pred)
        time3=time.time()
        train_time.append(time2-time1)
        test_time.append(time3-time2)
 
   
    class_dict={}
    classes=list(set(labels))
    for i in range(0,len(classes)):
        class_dict[classes[i]]=i
    print(class_dict)
    num_testLabels=[]
    for i in range(0,len(test_labels)):
        num_testLabels.append(class_dict[test_labels[i]])
       
    for i in range(0,len(y_pred)):
        test_labels1=list(test_labels)
        #Accuracy
       
        accuracy.append(accuracy_score(test_labels, y_pred[i]))
        
        #Precison
        precision.append(precision_score(test_labels, y_pred[i],average='macro'))
         
        #Recall
        Recall.append(recall_score(test_labels, y_pred[i],average='macro'))
           
        #f measure
        fMeasure.append(f1_score(test_labels, y_pred[i],average='macro'))
        
        #Precison
        mprecision.append(precision_score(test_labels, y_pred[i],average='micro'))
         
        #Recall
        mRecall.append(recall_score(test_labels, y_pred[i],average='micro'))
           
        #f measure
        mfMeasure.append(f1_score(test_labels, y_pred[i],average='micro'))
        
        kappa.append(cohen_kappa_score(test_labels, y_pred[i]))
      
        test = label_binarize(test_labels1, classes=categories).reshape((-1))
        pred = label_binarize( y_pred[i], classes=categories).reshape((-1))
        
        #roc
        try:
            roc.append(roc_auc_score(test,pred))
        except:
            roc.append(-1)
            
        #Average Mean Precision
        #y_val_true, val_pred = y_val_true.reshape((-1)), val_pred.reshape((-1))
        averageMeanPrecison.append(average_precision_score(test, pred))
        #MAE
        num_pred=[]
        for j in range(0,len(y_pred[i])):
            num_pred.append(class_dict[y_pred[i][j]])
        mae.append(mean_absolute_error(num_testLabels, num_pred))
        
        #PrintDetails(metric=metric,time=str(timedelta(seconds=(time2-time1))),measure="Average Mean Precison",Arr=averageMeanPrecison)


    
   #Accuracy
    PrintDetails(metric=metric,measure="Accuracy",Arr=accuracy,train_time=train_time,test_time=test_time)
    
    #Precison
    PrintDetails(metric=metric,measure="Precision",Arr=precision,train_time=train_time,test_time=test_time)
    #Precison
    PrintDetails(metric=metric,measure="micro Precision",Arr=mprecision,train_time=train_time,test_time=test_time)
    #Recall
    PrintDetails(metric=metric,measure="Recall",Arr=Recall,train_time=train_time,test_time=test_time)
     #Recall
    PrintDetails(metric=metric,measure="micro Recall",Arr=mRecall,train_time=train_time,test_time=test_time)
    #f measure
    PrintDetails(metric=metric,measure="F Measure",Arr=fMeasure,train_time=train_time,test_time=test_time)
    
    #f measure
    PrintDetails(metric=metric,measure="micro F Measure",Arr=mfMeasure,train_time=train_time,test_time=test_time)
    
    #roc
    PrintDetails(metric=metric,measure="ROC",Arr=roc,train_time=train_time,test_time=test_time)
    #Average Mean Precision
    PrintDetails(metric=metric,measure="Average Mean Precison",Arr=averageMeanPrecison,train_time=train_time,test_time=test_time)
    #MAE
    PrintDetails(metric=metric,measure="MAE",Arr=mae,train_time=train_time,test_time=test_time)
  
    accuracy_avg.append(accuracy)
    precision_avg.append(precision)
    recall_avg.append(Recall)
    macroFMeasure_avg.append(fMeasure)
    mprecision_avg.append(mprecision)
    mrecall_avg.append(mRecall)
    mFMeasure_avg.append(mfMeasure)
    roc_avg.append(roc)
    AMP_avg.append(averageMeanPrecison)
    mae_avg.append(mae)
    avg_test_time.append(test_time)
    avg_train_time.append(train_time)
    
import glob
datasets=[]
for file_name in glob.glob("Datasets April\\"+'*.csv'):
    file=os.path.split(file_name)[1]
    datasets.append(os.path.splitext(file)[0])

    
for dataset in datasets:
    fname="Datasets April\\"+dataset
    print(fname.encode())
    arr,labels=loadNonIRDatasets(fname)
    arr=np.array(arr)
    labels=np.array(labels)
    labels=list(labels)
    le = LabelEncoder()
    labels=le.fit_transform(labels)
    categories=list(set(labels))
    measures=["Euclidean"]
    global w
    w=0
    global Weights
    for met in measures:
        n_split=1
        labels=np.array(labels)
        path = "CKNN\\April Task1\\Cancelled Datasets"
        if not os.path.exists(path):
            os.makedirs(path)
        fname=path+'\\table_'+dataset+'_'+met+'_CKNN.txt'
        tables = open(fname, 'wb')
        k_splits=10
        accuracy_avg=[]
        precision_avg=[]
        recall_avg=[]
        macroFMeasure_avg=[]
        mprecision_avg=[]
        mrecall_avg=[]
        mFMeasure_avg=[]
        roc_avg=[]
        AMP_avg=[]
        mae_avg=[]
        avg_test_time=[]
        avg_train_time=[]
        kf = StratifiedKFold(n_splits=k_splits)
        for trn_ind, tst_ind in kf.split(arr,labels):
            xTrain, xTest = arr[trn_ind], arr[tst_ind]
            yTrain, yTest = labels[trn_ind], labels[tst_ind]
            train_data=np.array(xTrain)
            test_data=np.array(xTest)
            train_labels=np.array(yTrain)
            test_labels=np.array(yTest)
            allData=arr
            termOcc=[]
            docOcc=[]
            tables.write("\n*************".encode())
            tables.write((" Split\t"+str(n_split)).encode())
            tables.write(" ***********\n".encode())
            n_split=n_split+1
            k=1
            print("###############")
            Weights=np.ones(train_data.shape[0])
            classify(k=k,metric=met,termOccurance=termOcc,docOccurance=docOcc)
        list1=[5,15,30,45]
        tables.write("\n************* Average Results ***********\n".encode())
        acc_std=np.std(np.array(accuracy_avg),axis=0)
        accuracy_avg=np.mean(np.array(accuracy_avg),axis=0)
        
        pr_std=np.std(np.array(precision_avg),axis=0)
        precision_avg=np.mean(np.array(precision_avg),axis=0)
        
        mpr_std=np.std(np.array(mprecision_avg),axis=0)
        mprecision_avg=np.mean(np.array(mprecision_avg),axis=0)
        
        rcl_std=np.std(np.array(recall_avg),axis=0)
        recall_avg=np.mean(np.array(recall_avg),axis=0)
        
        mrcl_std=np.std(np.array(mrecall_avg),axis=0)
        mrecall_avg=np.mean(np.array(mrecall_avg),axis=0)
        
        F_std=np.std(np.array(macroFMeasure_avg),axis=0)
        macroFMeasure_avg=np.mean(np.array(macroFMeasure_avg),axis=0)
        
        mF_std=np.std(np.array(mFMeasure_avg),axis=0)
        mFMeasure_avg=np.mean(np.array(mFMeasure_avg),axis=0)
        
        roc_std=np.std(np.array(roc_avg),axis=0)
        roc_avg=np.mean(np.array(roc_avg),axis=0)
        
        amp_std=np.std(np.array(AMP_avg),axis=0)
        AMP_avg=np.mean(np.array(AMP_avg),axis=0)
        
        mae_std=np.std(np.array(mae_avg),axis=0)
        mae_avg=np.mean(np.array(mae_avg),axis=0)
        
        avg_train_time=np.mean(np.array(avg_train_time),axis=0)
        avg_test_time=np.mean(np.array(avg_test_time),axis=0)
        #Accuracy
        PrintDetails(metric=met,measure="Accuracy",Arr=accuracy_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=acc_std)
        
        #Precision
        PrintDetails(metric=met,measure="Precision",Arr=precision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=pr_std)
        
        #Precision
        PrintDetails(metric=met,measure="Micro Precision",Arr=mprecision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mpr_std)


        #Recall
        PrintDetails(metric=met,measure="Recall",Arr=recall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=rcl_std)
        
         #Recall
        PrintDetails(metric=met,measure="Micro Recall",Arr=mrecall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mrcl_std)


        #f measure
        PrintDetails(metric=met,measure="F Measure",Arr=macroFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=F_std)
        
       
       
        #f measure
        PrintDetails(metric=met,measure="Micro F Measure",Arr=mFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mF_std)


        #roc
        PrintDetails(metric=met,measure="ROC",Arr=roc_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=roc_std)

        #average Mean precision
        PrintDetails(metric=met,measure="AMP",Arr=AMP_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=amp_std)
         
        #MAE
        PrintDetails(metric=met,measure="MAE",Arr=mae_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mae_std)
        
        tables.close()
        print("###############")





C:\Users\Fix-Dell\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


b'Datasets April\\1_lung-cancer'
###############
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.02194070816040039
0.0030241012573242188
0.0029599666595458984
0.0009920597076416016
+----------+----------+----------------------+----------------------+----------------------+
|    k     | Accuracy |      Train Time      |      Test time       |      Total Time      |
+----------+----------+----------------------+----------------------+----------------------+
|    5     |   0.75   |    0:00:00.021941    |    0:00:00.182514    |    0:00:00.204455    |
|    15    |   0.75   |    0:00:00.003024    |    0:00:00.141619    |    0:00:00.144643    |
|    30    |   0.75   |    0:00:00.002960    |    0:00:00.142656    |    0:00:00.145616    |
|    45    |   0.75   |    0:00:00.000992    |    0:00:00.139629    |    0:00:00.140621    |
| Average  |   0.75   |    0:00:00.007229    |    0:00:00.151604    |    0:00:00.158834    |
| Std Dev. |   0.0    | 0.008532871734220931 | 0.017878706267924135 | 0.02

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.002991199493408203
0.0029921531677246094
0.003989219665527344
0.0029921531677246094
+----------+----------+------------------------+---------------------+----------------------+
|    k     | Accuracy |       Train Time       |      Test time      |      Total Time      |
+----------+----------+------------------------+---------------------+----------------------+
|    5     |   1.0    |     0:00:00.002991     |    0:00:00.136634   |    0:00:00.139626    |
|    15    |   1.0    |     0:00:00.002992     |    0:00:00.149600   |    0:00:00.152592    |
|    30    |   1.0    |     0:00:00.003989     |    0:00:00.135638   |    0:00:00.139627    |
|    45    |   1.0    |     0:00:00.002992     |    0:00:00.137632   |    0:00:00.140624    |
| Average  |   1.0    |     0:00:00.003241     |    0:00:00.139876   |    0:00:00.143117    |
| Std Dev. |   0.0    | 0.00043188028470191704 | 0.00565822813344995 | 0.005485496755301898 |
+----------+---------

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.0029921531677246094
0.001994609832763672
0.001997232437133789
0.003998279571533203
+----------+----------+-----------------------+----------------------+----------------------+
|    k     | Accuracy |       Train Time      |      Test time       |      Total Time      |
+----------+----------+-----------------------+----------------------+----------------------+
|    5     |  0.6667  |     0:00:00.002992    |    0:00:00.134640    |    0:00:00.137632    |
|    15    |  0.6667  |     0:00:00.001995    |    0:00:00.137669    |    0:00:00.139663    |
|    30    |  0.6667  |     0:00:00.001997    |    0:00:00.142622    |    0:00:00.144619    |
|    45    |  0.6667  |     0:00:00.003998    |    0:00:00.144636    |    0:00:00.148635    |
| Average  |  0.6667  |     0:00:00.002746    |    0:00:00.139892    |    0:00:00.142637    |
| Std Dev. |   0.0    | 0.0008297642451295927 | 0.003952667845354957 | 0.004295292332786748 |
+----------+----------

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.003030061721801758
0.0029582977294921875
0.0019931793212890625
0.0020248889923095703
+----------+----------+-----------------------+----------------------+----------------------+
|    k     | Accuracy |       Train Time      |      Test time       |      Total Time      |
+----------+----------+-----------------------+----------------------+----------------------+
|    5     |  0.3333  |     0:00:00.003030    |    0:00:00.139648    |    0:00:00.142678    |
|    15    |  0.6667  |     0:00:00.002958    |    0:00:00.151600    |    0:00:00.154558    |
|    30    |  0.6667  |     0:00:00.001993    |    0:00:00.144589    |    0:00:00.146582    |
|    45    |  0.6667  |     0:00:00.002025    |    0:00:00.149594    |    0:00:00.151619    |
| Average  |  0.5833  |     0:00:00.002502    |    0:00:00.146358    |    0:00:00.148859    |
| Std Dev. |  0.1443  | 0.0004933532149318556 | 0.004639468278524522 | 0.004568403437064532 |
+----------+--------

...

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.0064051151275634766
0.006452798843383789
0.0057675838470458984
0.005425214767456055
+----------+----------+------------------------+----------------------+----------------------+
|    k     | Accuracy |       Train Time       |      Test time       |      Total Time      |
+----------+----------+------------------------+----------------------+----------------------+
|    5     |  0.9636  |     0:00:00.006405     |    0:00:00.201457    |    0:00:00.207862    |
|    15    |   1.0    |     0:00:00.006453     |    0:00:00.202008    |    0:00:00.208461    |
|    30    |   1.0    |     0:00:00.005768     |    0:00:00.185914    |    0:00:00.191681    |
|    45    |   1.0    |     0:00:00.005425     |    0:00:00.227643    |    0:00:00.233068    |
| Average  |  0.9909  |     0:00:00.006013     |    0:00:00.204256    |    0:00:00.210268    |
| Std Dev. |  0.0157  | 0.00043384831717845654 | 0.014969028483626464 | 0.014784936087790817 |
+----------+

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.0
0.0
0.0
0.0
+----------+----------+------------+-----------------------+-----------------------+
|    k     | Accuracy | Train Time |       Test time       |       Total Time      |
+----------+----------+------------+-----------------------+-----------------------+
|    5     |  0.8909  |  0:00:00   |     0:00:00.207730    |     0:00:00.207730    |
|    15    |  0.9636  |  0:00:00   |     0:00:00.215871    |     0:00:00.215871    |
|    30    |  0.9636  |  0:00:00   |     0:00:00.215939    |     0:00:00.215939    |
|    45    |   1.0    |  0:00:00   |     0:00:00.216627    |     0:00:00.216627    |
| Average  |  0.9545  |  0:00:00   |     0:00:00.214042    |     0:00:00.214042    |
| Std Dev. |  0.0396  |    0.0     | 0.0036559712209105403 | 0.0036559712209105403 |
+----------+----------+------------+-----------------------+-----------------------+
metric Euclidean
measure Precision
0.0
0.0
0.0
0.0
+----------+-----------+------------

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.0
0.0
0.0
0.0
+----------+----------+------------+-----------------------+-----------------------+
|    k     | Accuracy | Train Time |       Test time       |       Total Time      |
+----------+----------+------------+-----------------------+-----------------------+
|    5     |  0.9636  |  0:00:00   |     0:00:00.216448    |     0:00:00.216448    |
|    15    |   1.0    |  0:00:00   |     0:00:00.215186    |     0:00:00.215186    |
|    30    |  0.9636  |  0:00:00   |     0:00:00.214754    |     0:00:00.214754    |
|    45    |  0.9636  |  0:00:00   |     0:00:00.217920    |     0:00:00.217920    |
| Average  |  0.9727  |  0:00:00   |     0:00:00.216077    |     0:00:00.216077    |
| Std Dev. |  0.0157  |    0.0     | 0.0012325366038064029 | 0.0012325366038064029 |
+----------+----------+------------+-----------------------+-----------------------+
metric Euclidean
measure Precision
0.0
0.0
0.0
0.0
+----------+-----------+------------

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.0016827583312988281
0.0
0.0
0.0
+----------+----------+----------------------+----------------------+-----------------------+
|    k     | Accuracy |      Train Time      |      Test time       |       Total Time      |
+----------+----------+----------------------+----------------------+-----------------------+
|    5     |  0.9455  |    0:00:00.001683    |    0:00:00.208138    |     0:00:00.209821    |
|    15    |   1.0    |       0:00:00        |    0:00:00.216442    |     0:00:00.216442    |
|    30    |   1.0    |       0:00:00        |    0:00:00.224580    |     0:00:00.224580    |
|    45    |   1.0    |       0:00:00        |    0:00:00.224741    |     0:00:00.224741    |
| Average  |  0.9864  |    0:00:00.000421    |    0:00:00.218475    |     0:00:00.218896    |
| Std Dev. |  0.0236  | 0.000728655731667348 | 0.006846691333091933 | 0.0062218115933228195 |
+----------+----------+----------------------+----------------------+----

{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.0
0.0
0.0
0.0049626827239990234
+----------+----------+----------------------+----------------------+---------------------+
|    k     | Accuracy |      Train Time      |      Test time       |      Total Time     |
+----------+----------+----------------------+----------------------+---------------------+
|    5     |   1.0    |       0:00:00        |    0:00:00.225043    |    0:00:00.225043   |
|    15    |   1.0    |       0:00:00        |    0:00:00.206817    |    0:00:00.206817   |
|    30    |  0.9636  |       0:00:00        |    0:00:00.218220    |    0:00:00.218220   |
|    45    |  0.9273  |    0:00:00.004963    |    0:00:00.227579    |    0:00:00.232541   |
| Average  |  0.9727  |    0:00:00.001241    |    0:00:00.219415    |    0:00:00.220656   |
| Std Dev. |  0.0302  | 0.002148904654952656 | 0.008038283871486923 | 0.00945986245587791 |
+----------+----------+----------------------+----------------------+---------------------+

In [2]:
"test"

'test'